# **Libraries**

In [ ]:
import pandas as pd
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
from skimage.transform import resize
from skimage.io import imread
import numpy as np
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix
import pickle
import seaborn as sns
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score, f1_score
import torchvision
import torchvision.models as models
from torch.utils.data.sampler import SubsetRandomSampler
import os
from torchvision import datasets, transforms
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
Categories=['trash','recycle','organics', 'hazardous']

# **Load Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train_dir="/content/drive/MyDrive/APS360/APS360 Project/APS360 Process Report Dataset/train"
val_dir="/content/drive/MyDrive/APS360/APS360 Project/APS360 Process Report Dataset/val"
test_dir="/content/drive/MyDrive/APS360/APS360 Project/APS360 Process Report Dataset/test"

In [ ]:
small_train_dir="/content/drive/MyDrive/APS360/APS360 Project/Process Report Small Dataset/train"
small_val_dir="/content/drive/MyDrive/APS360/APS360 Project/Process Report Small Dataset/val"

In [ ]:
torch.manual_seed(1)

# Apply transformations
transform = transforms.Compose([
    transforms.Resize(256),            # Resize to 256x256 pixels
    transforms.CenterCrop(224),        # Crop to 224x224 pixels
    # 224: standard input size for AlexNet, VGG, and ResNet
    # images to tensors: normalizes pixel values and allows for mathematical operations within NN
    transforms.ToTensor(),             # Convert images to tensors
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize with ImageNet mean and standard deviation
])

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

# Set up data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## For small dataset
small_train_dataset = datasets.ImageFolder(small_train_dir, transform=transform)
small_val_dataset = datasets.ImageFolder(small_val_dir, transform=transform)

# Set up data loaders
small_train_loader = DataLoader(small_train_dataset, batch_size=32, shuffle=True)
small_validation_loader = DataLoader(small_val_dataset, batch_size=32, shuffle=False)

# T**rain model on AlexNet**

In [ ]:
alexnet = models.alexnet(pretrained=True)
alexnet.classifier[6] = torch.nn.Linear(alexnet.classifier[6].in_features, 4)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
alexnet = alexnet.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:02<00:00, 85.7MB/s]


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class AlexNet(nn.Module):
    def __init__(self, num_classes=4):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

# Create the model
alexnet2 = AlexNet(num_classes=4)

In [ ]:
def train_net(model, train_loader, val_loader, batch_size=32,
              lr=0.001, num_epochs=15):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)  # Using Adam optimizer

    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        running_loss = 0.0
        train_preds = []
        train_targets = []

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            train_preds.extend(predicted.cpu().numpy())
            train_targets.extend(labels.cpu().numpy())

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = accuracy_score(train_targets, train_preds)
        epoch_f1 = f1_score(train_targets, train_preds, average='weighted')

        model.eval()  # Set model to evaluate mode
        val_running_loss = 0.0
        val_preds = []
        val_targets = []

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_running_loss += loss.item() * inputs.size(0)
                _, predicted = torch.max(outputs, 1)
                val_preds.extend(predicted.cpu().numpy())
                val_targets.extend(labels.cpu().numpy())

        val_loss = val_running_loss / len(val_loader.dataset)
        val_acc = accuracy_score(val_targets, val_preds)
        val_f1 = f1_score(val_targets, val_preds, average='weighted')

        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Training Accuracy: {epoch_acc:.4f}, F1-Score: {epoch_f1:.4f}, Loss: {epoch_loss:.4f}')
        print(f'Validation Accuracy: {val_acc:.4f}, F1-Score: {val_f1:.4f}, Loss: {val_loss:.4f}, ')


## Train on small dataset

In [ ]:
train_net(alexnet2, small_train_loader, small_validation_loader, batch_size=64,
              lr=0.0001, num_epochs=30)
'''we see that the training accuracy and f1-score is converging to 1.
but the model struggle with new or unseen data, since validation
metrics are not performing well.'''

Epoch 1/30
Training Accuracy: 0.4167, F1-Score: 0.3654, Loss: 1.2542
Validation Accuracy: 0.2500, F1-Score: 0.1623, Loss: 1.3278, 
Epoch 2/30
Training Accuracy: 0.4167, F1-Score: 0.3464, Loss: 1.2389
Validation Accuracy: 0.4167, F1-Score: 0.2778, Loss: 1.3166, 
Epoch 3/30
Training Accuracy: 0.5000, F1-Score: 0.4179, Loss: 1.1993
Validation Accuracy: 0.4167, F1-Score: 0.2917, Loss: 1.3070, 
Epoch 4/30
Training Accuracy: 0.3333, F1-Score: 0.2222, Loss: 1.2173
Validation Accuracy: 0.4167, F1-Score: 0.2917, Loss: 1.2980, 
Epoch 5/30
Training Accuracy: 0.5000, F1-Score: 0.4179, Loss: 1.1527
Validation Accuracy: 0.4167, F1-Score: 0.2917, Loss: 1.2920, 
Epoch 6/30
Training Accuracy: 0.4167, F1-Score: 0.3631, Loss: 1.2324
Validation Accuracy: 0.3333, F1-Score: 0.2381, Loss: 1.2845, 
Epoch 7/30
Training Accuracy: 0.5833, F1-Score: 0.5667, Loss: 1.1770
Validation Accuracy: 0.2500, F1-Score: 0.1667, Loss: 1.2754, 
Epoch 8/30
Training Accuracy: 0.5833, F1-Score: 0.5000, Loss: 1.0717
Validation Acc

## Train on original dataset

In [ ]:
'''these are the paramters that our team agreed on
for our progress report
Result:
Epoch 15/15
Training Accuracy: 0.6017, F1-Score: 0.5946, Loss: 0.9645
Validation Accuracy: 0.5700, F1-Score: 0.5417, Loss: 1.0353'''
train_net(alexnet2, train_loader, validation_loader, batch_size=32,
              lr=0.001, num_epochs=15)

Epoch 1/15
Training Accuracy: 0.3100, F1-Score: 0.2564, Loss: 1.6180
Validation Accuracy: 0.3200, F1-Score: 0.2684, Loss: 1.3583, 
Epoch 2/15
Training Accuracy: 0.3983, F1-Score: 0.3881, Loss: 1.2581
Validation Accuracy: 0.3350, F1-Score: 0.3173, Loss: 1.2847, 
Epoch 3/15
Training Accuracy: 0.5067, F1-Score: 0.5071, Loss: 1.1037
Validation Accuracy: 0.4400, F1-Score: 0.3661, Loss: 1.2238, 
Epoch 4/15
Training Accuracy: 0.4700, F1-Score: 0.4253, Loss: 1.3525
Validation Accuracy: 0.4250, F1-Score: 0.3607, Loss: 1.2961, 
Epoch 5/15
Training Accuracy: 0.4450, F1-Score: 0.4333, Loss: 1.1689
Validation Accuracy: 0.2900, F1-Score: 0.1730, Loss: 1.5422, 
Epoch 6/15
Training Accuracy: 0.4200, F1-Score: 0.4151, Loss: 1.2498
Validation Accuracy: 0.3750, F1-Score: 0.3775, Loss: 1.2588, 
Epoch 7/15
Training Accuracy: 0.5183, F1-Score: 0.4957, Loss: 1.0989
Validation Accuracy: 0.3950, F1-Score: 0.3406, Loss: 1.1958, 
Epoch 8/15
Training Accuracy: 0.5000, F1-Score: 0.4939, Loss: 1.0657
Validation Acc